In [10]:
import cv2
import numpy as np

# Data Collection and Preprocessing

In [11]:
# Load COCO class labels
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Model Loading and Training

In [12]:
# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Object Tracking using Optical FLow

In [13]:
# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Initialize variables for optical flow
old_frame = None
trackers = []

In [14]:
# Function to calculate Intersection over Union (IoU)
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    # Calculate coordinates of intersection area
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
    
    # Calculate area of intersection rectangle
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

    # Calculate area of both bounding boxes
    box1_area = w1 * h1
    box2_area = w2 * h2
    
    # Calculate IoU
    iou = inter_area / float(box1_area + box2_area - inter_area)
    
    return iou

# Real Time Object Detection

In [16]:
# Initialize video capture
cap = cv2.VideoCapture(0)  
while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape

    # Object detection using YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Lists to store detected objects
    detected_boxes = []
    confidences = []
    class_ids = []

    # Process each detection from YOLO output
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:  # Filter out weak detections
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                detected_boxes.append((x, y, w, h))
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Update optical flow trackers only if there are existing trackers and new detections
    if old_frame is not None and trackers and detected_boxes:
        # Convert frames to grayscale for optical flow
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

        for tracker in trackers:
            x, y, w, h = tracker["bbox"]
            roi = old_gray[y:y+h, x:x+w]
            p1, _, _ = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, np.array([[x + w // 2, y + h // 2]], dtype=np.float32), None, **lk_params)
            p1 = p1.reshape(-1, 2)

            # Update bounding box based on optical flow
            new_x = int(np.median(p1[:, 0]) - w // 2)
            new_y = int(np.median(p1[:, 1]) - h // 2)
            tracker["bbox"] = (new_x, new_y, w, h)

    # Initialize new trackers for new detections or update existing ones
    updated_trackers = []
    for box, confidence, class_id in zip(detected_boxes, confidences, class_ids):
        matched = False
        for tracker in trackers:
            iou = calculate_iou(tracker["bbox"], box)
            if iou > 0.3:  # IoU threshold to consider it as the same object
                # Update existing tracker
                tracker["bbox"] = box
                tracker["confidence"] = confidence
                tracker["class_id"] = class_id
                updated_trackers.append(tracker)
                matched = True
                break

        if not matched:
            # Create new tracker
            updated_trackers.append({"bbox": box, "confidence": confidence, "class_id": class_id})

    trackers = updated_trackers

    # Store current frame for the next iteration
    old_frame = frame.copy()

    # Display bounding boxes and labels from YOLO
    for tracker in trackers:
        x, y, w, h = tracker["bbox"]
        label = str(classes[tracker["class_id"]])
        color = (0, 255, 0)  # Green color for bounding box
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, f"{label} {tracker['confidence']:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Display optical flow tracking with red dot
        center_x = x + w // 2
        center_y = y + h // 2
        cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)  # Red dot for object center

    # Visualization
    cv2.imshow("Object Detection and Tracking with IoU", frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()
